In [1]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

TEST_DATASET= os.path.join('dataset', 'test.csv')
MODEL_DIR = 'models'

# Test Data Processing #

Before the model can be used to predict the test data, it has to be preprocessed, engineered and scaled just like during training.

In [2]:
test_dataset = pd.read_csv(TEST_DATASET)
df = test_dataset.copy()
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [3]:
ms_subclass_mapping = {20: 'MSSSubClass_20', 30: 'MSSSubClass_30', 40: 'MSSSubClass_40', 45: 'MSSSubClass_45', 
                       50: 'MSSSubClass_50', 60: 'MSSSubClass_60', 70: 'MSSSubClass_70', 75: 'MSSSubClass_75', 
                       80: 'MSSSubClass_80', 85: 'MSSSubClass_85', 90: 'MSSSubClass_90', 120: 'MSSSubClass_120', 
                       150: 'MSSSubClass_150', 160: 'MSSSubClass_160', 180: 'MSSSubClass_180', 
                       190: 'MSSSubClass_190'}
df['MSSubClass'] = df['MSSubClass'].map(ms_subclass_mapping)
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,MSSSubClass_20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,MSSSubClass_20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,MSSSubClass_60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,MSSSubClass_60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,MSSSubClass_120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [4]:
nan_columns = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 
              'GarageType',  'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']
for column in nan_columns:
    df[column].replace({np.nan: 'NA'}, inplace=True)
    
# Specifically replace MasVnrType nan values with None
df['MasVnrType'].replace({np.nan: 'None'}, inplace=True)
    
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,MSSSubClass_20,RH,80.0,11622,Pave,NA,Reg,Lvl,AllPub,...,120,0,NA,MnPrv,NA,0,6,2010,WD,Normal
1,1462,MSSSubClass_20,RL,81.0,14267,Pave,NA,IR1,Lvl,AllPub,...,0,0,NA,NA,Gar2,12500,6,2010,WD,Normal
2,1463,MSSSubClass_60,RL,74.0,13830,Pave,NA,IR1,Lvl,AllPub,...,0,0,NA,MnPrv,NA,0,3,2010,WD,Normal
3,1464,MSSSubClass_60,RL,78.0,9978,Pave,NA,IR1,Lvl,AllPub,...,0,0,NA,NA,NA,0,6,2010,WD,Normal
4,1465,MSSSubClass_120,RL,43.0,5005,Pave,NA,IR1,HLS,AllPub,...,144,0,NA,NA,NA,0,1,2010,WD,Normal


## Feature Engineering ##

In [5]:
ordinals = {
    'ExterQual': { 'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1 },
    'ExterCond': { 'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1 },
    'BsmtQual': { 'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0 },
    'BsmtCond': { 'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0 },
    'BsmtExposure': { 'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0 },
    'HeatingQC': { 'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1 },
    'KitchenQual': { 'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1 },
    'FireplaceQu': { 'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0 },
    'GarageFinish': { 'Fin': 3, 'RFn': 2, 'Unf': 1, 'NA': 0 },
    'GarageQual': { 'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0 },
    'GarageCond': { 'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0 },
    'PavedDrive': { 'Y': 3, 'P': 2, 'N': 1 },
    'PoolQC': { 'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'NA': 0 },
    'LotShape': { 'Reg': 4, 'IR1': 3, 'IR2': 2, 'IR3': 1 },
    'LandSlope': { 'Gtl': 3, 'Mod': 2, 'Sev': 1 },
    'CentralAir': { 'N': 0, 'Y': 1 },
    'Street': { 'Grvl': 1, 'Pave': 2 }
}

for key in list(ordinals.keys()):
    df[key] = df[key].map(ordinals[key])
    print('{} -> {}'.format(key, ordinals[key]))
    
df.head()

ExterQual -> {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}
ExterCond -> {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}
BsmtQual -> {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}
BsmtCond -> {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}
BsmtExposure -> {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0}
HeatingQC -> {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}
KitchenQual -> {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}
FireplaceQu -> {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}
GarageFinish -> {'Fin': 3, 'RFn': 2, 'Unf': 1, 'NA': 0}
GarageQual -> {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}
GarageCond -> {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}
PavedDrive -> {'Y': 3, 'P': 2, 'N': 1}
PoolQC -> {'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'NA': 0}
LotShape -> {'Reg': 4, 'IR1': 3, 'IR2': 2, 'IR3': 1}
LandSlope -> {'Gtl': 3, 'Mod': 2, 'Sev': 1}
CentralAir -> {'N': 0, 'Y': 1}
Street -> {'Grvl': 1, 'Pave': 2}


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,MSSSubClass_20,RH,80.0,11622,2,NA,4,Lvl,AllPub,...,120,0,0,MnPrv,NA,0,6,2010,WD,Normal
1,1462,MSSSubClass_20,RL,81.0,14267,2,NA,3,Lvl,AllPub,...,0,0,0,NA,Gar2,12500,6,2010,WD,Normal
2,1463,MSSSubClass_60,RL,74.0,13830,2,NA,3,Lvl,AllPub,...,0,0,0,MnPrv,NA,0,3,2010,WD,Normal
3,1464,MSSSubClass_60,RL,78.0,9978,2,NA,3,Lvl,AllPub,...,0,0,0,NA,NA,0,6,2010,WD,Normal
4,1465,MSSSubClass_120,RL,43.0,5005,2,NA,3,HLS,AllPub,...,144,0,0,NA,NA,0,1,2010,WD,Normal


In [6]:
df = pd.get_dummies(df, drop_first=True)
df.shape

(1459, 219)

In [7]:
df.head()

,Id,LotFrontage,LotArea,Street,LotShape,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1461,80.0,11622,2,4,3,5,6,1961,1961,...,0,0,0,0,1,0,0,0,1,0
1,1462,81.0,14267,2,3,3,6,6,1958,1958,...,0,0,0,0,1,0,0,0,1,0
2,1463,74.0,13830,2,3,3,5,5,1997,1998,...,0,0,0,0,1,0,0,0,1,0
3,1464,78.0,9978,2,3,3,6,6,1998,1998,...,0,0,0,0,1,0,0,0,1,0
4,1465,43.0,5005,2,3,3,8,5,1992,1992,...,0,0,0,0,1,0,0,0,1,0


In [8]:
def null_columns():
    columns = df.columns[df.isnull().any()]
    print(df[columns].isnull().sum())
    
null_columns()

LotFrontage     227
MasVnrArea       15
BsmtFinSF1        1
BsmtFinSF2        1
BsmtUnfSF         1
TotalBsmtSF       1
BsmtFullBath      2
BsmtHalfBath      2
KitchenQual       1
GarageYrBlt      78
GarageCars        1
GarageArea        1
dtype: int64


In [9]:
from scipy import stats

# LotFrontage
df['LotFrontage'].replace({np.nan: np.mean(df['LotFrontage'])}, inplace=True)

# MasVnrArea
df['MasVnrArea'].replace({np.nan: 0}, inplace=True)

# GarageYrBlt
mode, count = stats.mode(df['GarageYrBlt'], nan_policy='omit')
df['GarageYrBlt'].replace({np.nan: mode[0]}, inplace=True)

# Replace remaining NaN values with 0
df.replace({np.nan: 0}, inplace=True)

# Reset index
# df = df.reset_index(drop=True)

In [10]:
null_columns()

Series([], dtype: float64)


## Combining features ##

In [11]:
# 1.
df.drop(['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF'], axis=1, inplace=True)

# 2. 
df.drop(['1stFlrSF', '2ndFlrSF'], axis=1, inplace=True)

# 3. 
df['TotalBaths'] = df['BsmtFullBath'] + df['BsmtHalfBath'] + df['FullBath'] + df['HalfBath']
df.drop(['BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath'], axis=1, inplace=True)

# 4. 
df['GarageAreaPerCar'] = df['GarageArea'] / df['GarageCars']
df['GarageAreaPerCar'].replace({np.nan: 0}, inplace=True)  # Some values will equate to NaN because of 0 division
df.drop(['GarageCars', 'GarageArea'], axis=1, inplace=True)

# 5. 
df['TotalPorchSize'] = df['OpenPorchSF'] + df['EnclosedPorch'] + df['3SsnPorch'] + df['ScreenPorch']
df.drop(['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch'], axis=1, inplace=True)

df.head()

,Id,LotFrontage,LotArea,Street,LotShape,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,...,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,TotalBaths,GarageAreaPerCar,TotalPorchSize
0,1461,80.0,11622,2,4,3,5,6,1961,1961,...,0,1,0,0,0,1,0,1.0,730.0,120
1,1462,81.0,14267,2,3,3,6,6,1958,1958,...,0,1,0,0,0,1,0,2.0,312.0,36
2,1463,74.0,13830,2,3,3,5,5,1997,1998,...,0,1,0,0,0,1,0,3.0,241.0,34
3,1464,78.0,9978,2,3,3,6,6,1998,1998,...,0,1,0,0,0,1,0,3.0,235.0,36
4,1465,43.0,5005,2,3,3,8,5,1992,1992,...,0,1,0,0,0,1,0,2.0,253.0,226


## Feature Scaling ##

In [12]:
scale_columns = [ 'LotFrontage', 'LotArea', 'MasVnrArea', 'TotalBsmtSF', 'LowQualFinSF', 'GrLivArea', 
                 'TotalBaths', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageAreaPerCar', 
                 'WoodDeckSF', 'TotalPorchSize', 'PoolArea', 'MiscVal' ]

In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled = scaler.fit_transform(df[scale_columns])

# Make a new dataframe from returned scaled array
scaled_df = pd.DataFrame(scaled, columns=scale_columns)

# Drop previous columns from original dataframe and replace with scaled
# columns
df.drop(scale_columns, axis=1, inplace=True)
concatenated = [scaled_df, df]
df = pd.concat(concatenated, axis=1)
df.head()

,LotFrontage,LotArea,MasVnrArea,TotalBsmtSF,LowQualFinSF,GrLivArea,TotalBaths,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.555587,0.363929,-0.563316,-0.368484,-0.080483,-1.215588,-1.505575,-1.029543,-0.20391,-0.918335,...,0,0,0,0,1,0,0,0,1,0
1,0.604239,0.897861,0.047057,0.639542,-0.080483,-0.323539,-0.465502,0.175997,-0.20391,-0.255371,...,0,0,0,0,1,0,0,0,1,0
2,0.263676,0.809646,-0.563316,-0.264750,-0.080483,0.294508,0.574571,0.175997,-0.20391,-0.255371,...,0,0,0,0,1,0,0,0,1,0
3,0.458284,0.032064,-0.450284,-0.269260,-0.080483,0.243004,0.574571,0.175997,-0.20391,0.407593,...,0,0,0,0,1,0,0,0,1,0
4,-1.244533,-0.971808,-0.563316,0.529042,-0.080483,-0.424487,-0.465502,-1.029543,-0.20391,-0.918335,...,0,0,0,0,1,0,0,0,1,0


In [14]:
null_columns()

Series([], dtype: float64)


In [15]:
df.shape

(1459, 207)

# Predictions #

In [36]:
ids = df['Id']
features = df.drop('Id', axis=True)
print(ids.shape)
print(features.shape)

(1459,)
(1459, 206)


The final model will be used to predict on the important features that emerged during training.

In [37]:
feature_importance = ['LotArea', 'GrLivArea', 'OverallQual', 'TotalBsmtSF', 'TotalPorchSize', 'GarageAreaPerCar', 
                      'OverallCond', 'YearBuilt']
features = features[feature_importance]
features.head()

,LotArea,GrLivArea,OverallQual,TotalBsmtSF,TotalPorchSize,GarageAreaPerCar,OverallCond,YearBuilt
0,0.363929,-1.215588,5,-0.368484,0.258664,5.610600,6,1961
1,0.897861,-0.323539,6,0.639542,-0.501474,0.639982,6,1958
2,0.809646,0.294508,5,-0.264750,-0.519573,-0.204310,5,1997
3,0.032064,0.243004,6,-0.269260,-0.501474,-0.275658,6,1998
4,-0.971808,-0.424487,8,0.529042,1.217885,-0.061613,5,1992


In [62]:
from sklearn.externals import joblib

model = joblib.load(os.path.join(MODEL_DIR, 'model_final.joblib'))
pred = model.predict(features.values)

# Convert predictions from log to exponent representation
predictions = np.exp(pred)

data = { 'Id': ids, 'SalePrice': predictions }
prediction_df = pd.DataFrame(data=data, columns=['Id', 'SalePrice'], index=np.arange(predictions.shape[0]))
prediction_df.head()

,Id,SalePrice
0,1461,145121.572799
1,1462,174150.683883
2,1463,168829.469027
3,1464,188753.605203
4,1465,199549.260220


# Write predictions to CSV #

In [61]:
if not os.path.exists('predictions'):
    os.mkdir('predictions')

filepath='predictions/predictions.csv'
prediction_df.to_csv(filepath, index=False)